### The purpose of this notebook is to load the data via POST requests

#### Import depedencies

In [64]:
import requests
import pandas as pd
import numpy as np
import datetime
import json

### Get setup variables

In [68]:
with open('config.json') as json_file:
    data = json.load(json_file)
    BASE_URL  = data['EC2_API_ENDPOINT']

#### Load cleaned product data

In [2]:
dataFile = '../data/product_data.xlsx'

In [6]:
data = pd.read_excel(dataFile, sheet_name = 'all_prods', parse_dates=True)

#### View data

In [7]:
data.head()

,raw_product,brand,category,name,slug,description,color,is available,qty,retail price,sale price,release date
0,Adidas-Yeezy-Boost-350-Low-V2-Beluga,adidas,footwear,adidas yeezy boost 350,adidas-yeezy-boost-350-low-v2-beluga,"low, v2, beluga",NaN,1,498,220,1097,2016-09-24
1,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,adidas,footwear,adidas yeezy boost 350,adidas-yeezy-boost-350-v2-core-black-copper,"v2, core","black,copper",1,225,220,685,2016-11-23
2,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,adidas,footwear,adidas yeezy boost 350,adidas-yeezy-boost-350-v2-core-black-green,"v2, core","black, green",1,196,220,690,2016-11-23
3,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,adidas,footwear,adidas yeezy boost 350,adidas-yeezy-boost-350-v2-core-black-red,"v2, core","black, red",1,302,220,1075,2016-11-23
4,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,adidas,footwear,adidas yeezy boost 350,adidas-yeezy-boost-350-v2-core-black-red-2017,"core, 2017","black,red",1,969,220,828,2017-02-11


### Create URL and credentials for REST API

In [69]:
base_url = BASE_URL

### Step 1: Create new user

#### Create Guest user and return the token needed for authentifcation

In [18]:
EMAIL = "guest@awit.com"
NAME = "Guest user"
PASSWORD = "bemyguest123"

In [13]:
DATA = {
    "email": "guest@awit.com",
    "name": "Guest user",
    "password": "bemyguest123"
}

In [71]:
query_url = f"{base_url}profile/"
#print(query_url)

In [15]:
response = requests.post(url=query_url, data=DATA).json()

In [16]:
print(response)

{'id': 2, 'email': 'guest@awit.com', 'name': 'Guest user'}


### Step 2: Login with new user

In [72]:
query_url = f"{base_url}login/"
#print(query_url)

In [19]:
DATA = {
    'username': EMAIL, 
    'password': PASSWORD
}

In [20]:
response = requests.post(url=query_url, data=DATA).json()

In [ ]:
MY_TOKEN = response['token']

### Step 3: Set header dict for posting data

In [22]:
HEADER = {'Authorization':f'Token {MY_TOKEN}'}

## Section to Load Data

### Step 1: Load brands

In [24]:
ALL_BRANDS = data['brand'].unique()
ALL_BRANDS

array(['adidas', 'nike', 'jordan'], dtype=object)

In [74]:
query_url = f"{base_url}brand/"
#print(query_url)

In [29]:
for brand in ALL_BRANDS:
    load_data = {'name': brand, 'is_active': True}
    response = requests.post(url=query_url, headers=HEADER, data=load_data).json()
    print(response)

{'id': 1, 'name': 'adidas', 'is_active': True}
{'id': 2, 'name': 'nike', 'is_active': True}
{'id': 3, 'name': 'jordan', 'is_active': True}


## View all loaded brands

In [31]:
db_brands = requests.get(url=query_url, headers=HEADER).json()
print(db_brands)

[{'id': 1, 'name': 'adidas', 'is_active': True}, {'id': 2, 'name': 'nike', 'is_active': True}, {'id': 3, 'name': 'jordan', 'is_active': True}]


### Step 2: Load Category

In [32]:
ALL_CATS = data['category'].unique()
ALL_CATS

array(['footwear'], dtype=object)

In [75]:
query_url = f"{base_url}category/"
#print(query_url)

In [34]:
for cat in ALL_CATS:
    load_data = {'name': cat, 'is_active': True}
    response = requests.post(url=query_url, headers=HEADER, data=load_data).json()
    print(response)

{'id': 1, 'name': 'footwear', 'is_active': True}


## View all loaded categories

In [35]:
db_cats = requests.get(url=query_url, headers=HEADER).json()
print(db_cats)

[{'id': 1, 'name': 'footwear', 'is_active': True}]


### Step 3: Load Product

In [76]:
query_url = f"{base_url}product/"
#print(query_url)

In [39]:
cols_imp = ['brand','category','slug',
            'raw_product','retail price','sale price',
            'release date','qty','is available',
                    ]
cols_imp

['brand',
 'category',
 'slug',
 'raw_product',
 'retail price',
 'sale price',
 'release date',
 'qty',
 'is available']

In [40]:
db_load_data = data[cols_imp]
db_load_data.head(3)

,brand,category,slug,raw_product,retail price,sale price,release date,qty,is available
0,adidas,footwear,adidas-yeezy-boost-350-low-v2-beluga,Adidas-Yeezy-Boost-350-Low-V2-Beluga,220,1097,2016-09-24,498,1
1,adidas,footwear,adidas-yeezy-boost-350-v2-core-black-copper,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,220,685,2016-11-23,225,1
2,adidas,footwear,adidas-yeezy-boost-350-v2-core-black-green,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,220,690,2016-11-23,196,1


In [51]:
db_load_data.dtypes

brand                   object
category                object
slug                    object
raw_product             object
retail price             int64
sale price               int64
release date    datetime64[ns]
qty                      int64
is available             int64
dtype: object

In [58]:
db_load_data['updated_release_date_format'] = db_load_data['release date'].dt.strftime('%Y-%m-%d')
db_load_data.head(3)

<ipython-input-58-b70b1927ad8a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_load_data['updated_release_date_format'] = db_load_data['release date'].dt.strftime('%Y-%m-%d')


,brand,category,slug,raw_product,retail price,sale price,release date,qty,is available,updated_release_date_format
0,adidas,footwear,adidas-yeezy-boost-350-low-v2-beluga,Adidas-Yeezy-Boost-350-Low-V2-Beluga,220,1097,2016-09-24,498,1,2016-09-24
1,adidas,footwear,adidas-yeezy-boost-350-v2-core-black-copper,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,220,685,2016-11-23,225,1,2016-11-23
2,adidas,footwear,adidas-yeezy-boost-350-v2-core-black-green,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,220,690,2016-11-23,196,1,2016-11-23


In [59]:
db_load_data.dtypes

brand                                  object
category                               object
slug                                   object
raw_product                            object
retail price                            int64
sale price                              int64
release date                   datetime64[ns]
qty                                     int64
is available                            int64
updated_release_date_format            object
dtype: object

In [62]:
for index,row in db_load_data.iterrows():
    name = row['raw_product'].replace("-"," ").lower().strip()
    brand_raw = row['brand']
    category = 1
    slug = row['slug']
    retail_price = row['retail price']
    sale_price = row['sale price']
    qty = row['qty']
    release_date = row['updated_release_date_format']
    
    
    if brand_raw == 'adidas':
        brand_id = 1
    elif brand_raw == 'nike':
        brand_id = 2
    elif brand_raw == 'jordan':
        brand_id = 3
        
    load_input = {
        "name": name,
        "slug": slug,
        "category":category,
        "brand":brand_id ,
        "retail_price": retail_price,
        "sale_price": sale_price,
        "qty": qty,
        "release_date": release_date,
        "is_available": True
    }
    #print(load_input)
    response = requests.post(url=query_url, headers=HEADER, data=load_input).json()
    print(response)

## View all loaded products

In [63]:
db_prods = requests.get(url=query_url, headers=HEADER).json()
print(db_prods)

[{'id': 1, 'name': 'adidas yeezy boost 350 low v2 beluga', 'slug': 'adidas-yeezy-boost-350-low-v2-beluga', 'category': 1, 'brand': 1, 'retail_price': '220.00', 'sale_price': '1097.00', 'qty': 498, 'release_date': '2016-09-24', 'is_available': True}, {'id': 2, 'name': 'adidas yeezy boost 350 v2 core black copper', 'slug': 'adidas-yeezy-boost-350-v2-core-black-copper', 'category': 1, 'brand': 1, 'retail_price': '220.00', 'sale_price': '685.00', 'qty': 225, 'release_date': '2016-11-23', 'is_available': True}, {'id': 3, 'name': 'adidas yeezy boost 350 v2 core black green', 'slug': 'adidas-yeezy-boost-350-v2-core-black-green', 'category': 1, 'brand': 1, 'retail_price': '220.00', 'sale_price': '690.00', 'qty': 196, 'release_date': '2016-11-23', 'is_available': True}, {'id': 4, 'name': 'adidas yeezy boost 350 v2 core black red', 'slug': 'adidas-yeezy-boost-350-v2-core-black-red', 'category': 1, 'brand': 1, 'retail_price': '220.00', 'sale_price': '1075.00', 'qty': 302, 'release_date': '2016-11